# 🦠 Visualização de Dados da COVID-19

## 🏁 Introdução
Utilização de dados do **AWS COVID-19 data lake** para analisar informações precisas sobre a disseminação e as características do coronavírus (SARS-CoV-2). 

Com esses dados, há a apresentação de diversas visualizações, explorando várias correlações entre eles. Destacam-se o **gráfico de barras**, que compara a disponibilidade de leitos hospitalares entre diferentes regiões, e o **mapa interativo da linha do tempo**, que mostra a evolução de mortes mundialmente. 

O objetivo principal é compreender as informações obtidas no data lake e apresentar as visualizações mais eficazes para gerar **insights pertinentes**. 

## 📚Importação das bibliotecas

In [2]:
import pandas as pd
import plotly.express as px

## 📖 Leitura dos dados

In [19]:
USA_STATES_HOSPITAL_BEDS_PATH = 'data_for_visualization_csv/usa_states_hospital_beds.csv'
usa_states_hospital_beds_df = pd.read_csv(USA_STATES_HOSPITAL_BEDS_PATH)


USA_CASES_AND_DEATHS_PATH = 'data_for_visualization_csv/usa_states_cases_and_deaths.csv' 
usa_states_cases_and_deaths_df = pd.read_csv(USA_CASES_AND_DEATHS_PATH)

GLOBAL_DEATHS_PATH = 'data_for_visualization_csv/global_deaths.csv'
global_deaths_df = pd.read_csv(GLOBAL_DEATHS_PATH)

USA_STATES_VACCINATED_PER_HUNDRED_PATH = 'data_for_visualization_csv/usa_states_vaccinated_per_hundred.csv' 
usa_states_vaccinated_per_hundred_df = pd.read_csv(USA_STATES_VACCINATED_PER_HUNDRED_PATH)

USA_STATES_DOSES_PFIZER_VACCINE_PATH = 'data_for_visualization_csv/usa_states_doses_pfizer_vaccine.csv' 
usa_states_doses_pfizer_vaccine_df = pd.read_csv(USA_STATES_DOSES_PFIZER_VACCINE_PATH)

## 👓Análises e Visualizações
### 1. Estados Americanos e camas disponíveis

### 2. Estados Americanos: Casos e Mortes

### 3. Mortes mundialmente

### 4. Estados Americanos e a média de vacinas por 100 habitantes 
#### Mesclado com os maiores votos para partidos eleitorais nas eleições de 2020

### 5. Estados Americanos e a Primeira e Segunda Dose da Vacina Pfizer

## 🏁Conclusão